# OpenCV object tracking
In this notebook we track objects in videos by means of opencv.

## Imports
Includes our own libraries

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.imageutil import show_image, median_canny

## Optical flow
https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html

In [32]:
capture = cv2.VideoCapture('../video/walking-people.avi')

width = int(capture.get(3)) 
height = int(capture.get(4))

print(f'width: {width}, height: {height}')

# we need to write a modified video back
out = cv2.VideoWriter('./out/optical_flow_walking_people.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

# use random colors to create trails for object movement in the image 
color = np.random.randint(100, 255, (100, 3))

# find corners in first frame
ret, prev_frame = capture.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

feature_params = dict( maxCorners = 200, qualityLevel = 0.5, minDistance = 9, blockSize = 7 )
prev_features = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)

# create a mask image for drawing purposes
mask = np.zeros_like(prev_frame)

# set parameters for optical flow
optical_flow_params = dict( winSize  = (15,15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

while(True):
    ret, frame = capture.read()

    if ret == True:
      frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      new_features, status, errors = cv2.calcOpticalFlowPyrLK(prev_gray, 
                                                            frame_gray, 
                                                            prev_features, 
                                                            None, 
                                                            **optical_flow_params)

      # select and store good points
      good_new = new_features[status==1]
      good_old = prev_features[status==1]

      # draw the tracks
      for i,(new,old) in enumerate(zip(good_new, good_old)):
          a, b = new.astype(int).flatten()
          c, d = old.astype(int).flatten()
          mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
          frame = cv2.circle(frame, (a,b), 5, color[i].tolist(),-1)
          
      out.write(cv2.add(frame, mask))
        
      # update the previous frame and previous points
      prev_gray = frame_gray.copy()
      prev_features = good_new.reshape(-1,1,2)

    else:
      break
    
capture.release()
out.release()

width: 960, height: 540


In [33]:
!ffmpeg -i ./out/optical_flow_walking_people.avi ./out/optical_flow_walking_people.mp4 -y

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-0)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1687154882217/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1687154882217/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1687154882217/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1687154882217/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1687154882217/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav

In [34]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('/out/optical_flow_walking_people.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)